In [53]:
import pysam
from collections import *
from tools.intervals import *
from tools.transcripts import *
from tools.fileOps import *
from tools.bio import *
from tools.psl import *
from tools.procOps import *
from orderedset import OrderedSet

In [2]:
fq_with_barcodes = '/hive/users/ifiddes/notch2nl_berkeley_data/E2del19N_E2del68_combined_longranger/E2del68_E2del19N_combined/E2del68_E2del19N_combined.sorted.fastq'

In [12]:
bcode_map = {}
for x in open(fq_with_barcodes):
    if x.startswith('@'):
        n, t = x.split()
        n = n[1:-2]  # remove identifier
        bcode_map[n] = t

In [20]:
# for each bam, organize reads by barcode
reads_by_bcode = defaultdict(lambda: defaultdict(list))
for n in ['A1', 'A2', 'B1', 'B2', 'C1', 'C2', 'D1', 'D2', 'N1', 'N2']:
    for rec in pysam.Samfile(n + '.binreads.bam'):
        if rec.is_unmapped:
            continue
        bcode = bcode_map[rec.qname]
        reads_by_bcode[n][bcode].append(rec)

In [ ]:
# construct a set of intervals for each barcode
# also maintain a linear map of sequences to show indels
bcode_intervals = defaultdict(dict)
for n in reads_by_bcode:
    for bcode, recs in reads_by_bcode[n].iteritems():
        intervals = []
        seqs = []
        for rec in recs:
            for block in rec.blocks

In [44]:
# try using blat
# for each set of recs, if they are the primary mapping, concatenate the sequence in linear ordering
bcode_seqs = defaultdict(dict)
for n in reads_by_bcode:
    for bcode, recs in reads_by_bcode[n].iteritems():
        seq = OrderedSet()
        for rec in recs:
            if not rec.is_secondary and not rec.is_supplementary:
                seq.add(rec.seq)
        bcode_seqs[n][bcode] = ''.join(seq)

In [66]:
alns = defaultdict(list)
with TemporaryFilePath() as tmp_fa, TemporaryFilePath() as tmp_psl:
    for n in bcode_seqs:
        for bcode, seq in bcode_seqs[n].iteritems():
            with open(tmp_fa, 'w') as outf:
                write_fasta(outf, bcode, seq)
            cmd = ['blat', '../{}.fa'.format(n), '-noHead', tmp_fa, tmp_psl]
            run_proc(cmd)
            cmd = ['simpleChain', tmp_psl, '/dev/stdout', '-outPsl']
            r = call_proc_lines(cmd)
            recs = [PslRow(x.split()) for x in r]
            rec = sorted(recs, key=lambda x: x.coverage)[-1]
            alns[n].append(rec)